## Load model

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../gw")

In [6]:
from generation_wrapper import RuGPT3XL

Note! seq_len is max sequence length for generation used in generation process. Max avialable seq_len is 2048 (in tokens).
Also inference takes around 10 Gb GPU memory.

In [7]:
gpt = RuGPT3XL.from_pretrained("sberbank-ai/rugpt3xl", seq_len=512)

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
Use alternating sparse & dense attention layers


#### Get logits

In [10]:
logits = gpt("Кто был президентом США в 2020? ").logits

In [11]:
type(logits), logits.shape

(torch.Tensor, torch.Size([1, 8, 50264]))

#### Get loss

In [12]:
input_ids = [gpt.tokenizer("Кто был президентом США в 2020? ")['input_ids']]
labels = input_ids

In [13]:
import torch


with torch.no_grad():
    loss = gpt(input_ids=input_ids, labels=labels).loss

In [14]:
loss

[tensor(4.3871, device='cuda:0')]

#### Simple generation

In [18]:
def filter_resuls(nr):
    return [x[:x.find("<|endoftext|>")] for x in nr]

Greedy decoding

In [19]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ",
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nВ этом году выборы президента Соединенных Штатов Америки пройдут уже через несколько дней. И, как и всегда на протяжении последних лет (а это более чем 20-ти), кандидаты будут бороться за право стать главой государств']

sample

In [21]:
filter_resuls(gpt.generate(
    "Кто был президентом США в 2020? ", do_sample=True, num_return_sequences=5,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Кто был президентом США в 2020? \nВ следующем десятилетии президентство займет человек, который появится на свет 18 января. За это время у него будут все шансы не только достойно войти во взрослый мир политики и даже стать главой государства (для этого ег',
 'Кто был президентом США в 2020? \nИлья Мельчук (КПРФ) – 506 голосов за 1% + 411 депутатов, 2-е место по явке. Не удалось стать кандидатом от СР во втором туре выборов президента Р',
 'Кто был президентом США в 2020? \nСейчас я предлагаю подумать о том, кто бы исполнял обязанности президента России следующие два года. \n Вы за или против того чтобы на этот пост встал Дмитрий Медведев!? Думаю это вопрос не праздный и здесь ест',
 'Кто был президентом США в 2020? \nПрезидентской избирательной кампанией во многих странах мира все чаще начинает руководить не сам президент, а партийцы. В 2016 году на этот пост баллотировался кандидат от Демократической партии Джеб Буш-мла',
 'Кто был президентом США в 2020? \nЗависит ли выбор следующе

#### Top_k top_p filtering

In [26]:
filter_resuls(gpt.generate(
    "Александр Сергеевич Пушкин родился в ",
    top_k=5,
    top_p=0.95,
    temperature=1.2,
    num_return_sequences=5,
    do_sample=True,
    max_length=50,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
))

['Александр Сергеевич Пушкин родился в \n1799 г. и прожил до 28 лет (1821-27), когда, по словам Вигеля, «умер на дуэли». Он был сыном богатого помещика Сергея Львовича Пушкина от брака ег',
 'Александр Сергеевич Пушкин родился в \n1812 году и умер через десять лет, не оставив потомства. В 1830-м его мать вышла замуж за богатого помещика Александра Ивановича Тургенева (1787—1936) — брата матери Пушкин',
 'Александр Сергеевич Пушкин родился в \xa01799 году, то есть ему было всего лишь 18 лет. Но уже через два года он был зачислен юнкером-артиллеристом во 2й егерский полк и начал службу подпор',
 'Александр Сергеевич Пушкин родился в \n1799 году (по другим данным — 1820). Его родители: отец, отставной поручик артиллерии Александр Иванович Пущин; мать - Екатерина Андреевна Карамзина. В семье Пушкина было четверо детей ',
 'Александр Сергеевич Пушкин родился в \n1899 г. на юге России, где провел свои детские годы и юность (родной хутор деда его - Ганнибала Пушкина), а с 1920 года жил во вре

#### Beamsearch

In [27]:
filter_resuls(gpt.generate(
    text="Александр Сергеевич Пушкин родился в ",
    max_length=50,
    num_beams=10,
    no_repeat_ngram_size=3,
    repetition_penalty=2.,
    num_return_sequences=5,
))

['Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урожденная Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Мария Алексеевна Ганнибал',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать поэта, Наталья Николаевна \nПушкин',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \nдворянского рода. Мать, урождённая Ганнибал, был',
 'Александр Сергеевич Пушкин родился в \n1799 году в селе Михайловском Псковской губернии. Его отец, \nАлександр Львович Пушкин, происходил из старинного \n